In [1]:
import tweepy
import json
import pandas as pd
import random
from dotenv import load_dotenv
import os
import time
from datetime import datetime
load_dotenv()

True

# Sentiment analysis de tweets respecto al plebiscito en Septiembre 2022 Chile

Este script busca en tweeter, con la los privilegios de la cuenta develop gratuita, los tweets según las palabras clave, la localización y las fechas determinadas.

La información mas importante de cada tweet es guardado en un dataframe.

La API libre de twitter tiene varias limitaciones, entre ella la fecha:

    Keep in mind that the search index has a 7-day limit. In other words, no tweets will be found for a date older than one week.
    
No se puede traer tweets mas antiguos que una semana. Ademas del peso sobre la cantidad de tweets, que cuelga la API por 15 minutos

##### Definición de palabras clave (_keywords_) a buscar dentro del texto de los tweets:

In [2]:
keywords_list = ['Constituyente',
'Convención Constituyente',
'Convención Constitucional',
'Asamblea Constituyente',
'plebiscito',
'referéndum',
'constitución chilena',
'Acuerdo por la paz social y la Nueva Constitución',
'Vamos por Chile',
'Unión Demócrata Independiente',
'Constituyentes Apartidistas',
'Independientes No Neutrales',
'Lista del Pueblo',
'Apruebo Dignidad',
'Lista del Apruebo',
'Pueblo Constituyente']

keywords_list_II = ['Chile Digno',
'Chile Unido',
'Unidos por Chile'
'Chile Libre',
'Coordinadora Constituyente Plurinacional y Popular',
'Somos región',
'Colectivo Socialista',
'Independientes por una Nueva Constitución',
'Movimientos sociales constituyentes',
'Colectivo del Apruebo',
'Wallmapur',
'Elisa Loncón',
'Campaña de desprestigio',
'rechazo del prebiscito de 2020',
'Carmen Gloria Valladares Moyano',
'Elisa Loncon',
'Destitucion de Elisa Loncon']

In [3]:
keywords_list = ["(" + word + ")" for word in keywords_list]

'#DestitucionDeElisaLoncon',
'juicio político',
'Jaime Bassa',
'Marcela Cubillos',
'Jaime Mañalich',
'Autodesprestigio',
'María Elisa Quinteros Cáceres',
'Gaspar Dominguez Donoso']

In [4]:
keywords_list  = ' OR '.join(keywords_list)

In [5]:
keywords_list

'(Constituyente) OR (Convención Constituyente) OR (Convención Constitucional) OR (Asamblea Constituyente) OR (plebiscito) OR (referéndum) OR (constitución chilena) OR (Acuerdo por la paz social y la Nueva Constitución) OR (Vamos por Chile) OR (Unión Demócrata Independiente) OR (Constituyentes Apartidistas) OR (Independientes No Neutrales) OR (Lista del Pueblo) OR (Apruebo Dignidad) OR (Lista del Apruebo) OR (Pueblo Constituyente)'

#### Funciones
El script tiene dos funciones principales:
- Activación de la API con las credenciales
- Búsqueda de los tweets con lo parámetros definidos

##### Función Activación

In [6]:
# Función de generación de la API
def oauth():
    #Pass in our twitter API authentication key
    consumer_key = os.getenv('CONSUMER_KEY')
    consumer_secret = os.getenv('CONSUMER_SECRET')
    access_token = os.getenv('ACCESS_TOKEN')
    access_token_secret = os.getenv('ACCESS_TOKEN_SECRET')
    auth = tweepy.OAuth1UserHandler(
        consumer_key, consumer_secret,
        access_token, access_token_secret
    )
#     Instantiate the tweepy API
    api = tweepy.API(auth, wait_on_rate_limit=True)
    return api

##### Función Búsqueda

In [7]:
# Función de búsqueda de tweets según los keywords, cantidad y localización
def search(geo, search_query, page_limit):
    page_no = 1
    # Creamos las columnas para el DataFrame
    data = []
    dict = {}
    page_no = 1

    df = pd.DataFrame()
    columns = ['Time','User', 'Tweet']
    try:
        # Lanzamos la búsqueda
        pages = tweepy.Cursor(api.search_tweets,
                              q=search_query, 
                              tweet_mode='extended', 
                              geocode = geo).pages(page_limit)
        print(pages)
    # Iterate through the pages and print the text of the tweets
        for page in pages:    
            for i in range(len(page)):
                data.append([page[i].created_at, page[i].user.screen_name, page[i].full_text])
            page_no += 1
        df = pd.DataFrame(data, columns=columns)
        return df
    
    except BaseException as e:
        print('Status Failed On,',str(e))
        return

#### Ejecución 

##### Instanciamos la API

In [8]:
api = oauth()
api

#####  Cargamos los parámetros de búsqueda


- `sgo`: latitud y longitud de Santiago de Chile
- `radio`: radio en km alrededor del punto
- `no_of_tweets`: Cantidad de tweets para cada keyword
- `geo`: arma el feature según lo cargado anteriormente, con la ciudad y el radio.
- `keywords_list`: Se incluye en esta sección sólo para testear. Se limita a dos palabras. Si se comenta esa línea, usa toda la lista presentada arriba)

In [9]:
sgo = '-33.4489,-70.6693'
radio = '100km'
page_limit =2
geo = str(sgo) + ',' + str(radio) 
# keywords_list = ['Chile'] # Comentar esta linea para usar toda la keywords_list


In [10]:
keywords_list

'(Constituyente) OR (Convención Constituyente) OR (Convención Constitucional) OR (Asamblea Constituyente) OR (plebiscito) OR (referéndum) OR (constitución chilena) OR (Acuerdo por la paz social y la Nueva Constitución) OR (Vamos por Chile) OR (Unión Demócrata Independiente) OR (Constituyentes Apartidistas) OR (Independientes No Neutrales) OR (Lista del Pueblo) OR (Apruebo Dignidad) OR (Lista del Apruebo) OR (Pueblo Constituyente)'

#####  Lanzamos la búsqueda

In [11]:
# df = pd.DataFrame()
df = search(geo,
            keywords_list,
            page_limit)


In [12]:
pd.set_option('display.max_colwidth', None)
df

,Time,User,Tweet
0,2022-08-24 18:10:57,RadioUniverso,"🎙 Esta mañana la Ministra de la Mujer y Equidad de Género, Antonia Orellana @totiorellanag en #UniversoAlDía se refirió al rol que ha tenido el gobierno de informar sobre el próximo plebiscito constituyente.\n\n👉🏽Revive esta entrevista completa en https://t.co/abzHAYIXxJ https://t.co/BTcqIsXT4j"
1,2022-08-24 18:10:56,el_villano_va,@ingridna740 @latercera Pasando el plebiscito lo enjuicia la justicia mapudungún que aún no se termina de ✍️ escribir :))
2,2022-08-24 18:10:47,marcosconba,"@vladomirosevic No se adelanten, esperemos que la CAM no lo considere un agravio, semanas antes del plebiscito"
3,2022-08-24 18:09:40,patotenia,"@_renzuc No creo que la gente, especialmente la de la macrozona, modifique su voto por esto. Si no hay represalias de la Cam antes del plebiscito, esto es un tongo."
4,2022-08-24 18:09:12,mauriciokegan,"@Iqqdemicorazon Ojalá sea cierto y no otra movida desesperada del Gobierno ara tratar de revertir el resultado del plebiscito logrando votos para el apruebo, para luego inventar otra Boricosas y liberarlo."
5,2022-08-24 18:09:08,wchgrafica,"Estallido Social-Acuerdo 1/2cocinado, pero acuerdo al fin- Elección: Plebiscito/Asamblea-80% vs 20% Convencionales +de 2/3 -Texto Propuesto. Todo democráticamente x voluntad del pueblo. Solo nos queda votar APRUEBO y cerramos el círculo. NO MAS DERECHA y sus privilegios,MIENTEN."
6,2022-08-24 18:08:52,1diegow,"@PamJiles @biobio Para su información, Ud no acumula ningún poder en este plebiscito. Su voto no vale más que el de nadie, no hay nada que distribuir.\n\nOjalá todos votemos informados y con convicción. Lástima que usted no sea un ejemplo de ello."
7,2022-08-24 18:08:20,rositapaez,"Lo de Llaitul será un tongo? \nDe ser así, saldrá el lunes 5 a las 8 am. De no ser así va a quedar la escoba en la Araucanía. Yo no tengo buenos presentimientos. Creo que algo muy malo va a suceder antes del plebiscito. Será este el motivo? La ultraizquierda es maligna."
8,2022-08-24 18:08:20,javierpevec,@gonzalox214 @vladomirosevic Pero si la querella la presento Piñera. Que tiene que ver el plebiscito acá
9,2022-08-24 18:08:10,elias_b_l,"@DeRegreso5 Como hacer entender a esta gente que ser de izquierda no significa que debes vivir al 3 y al 4, y que en el plebiscito votan chilenos habilitados estén o no en chile"


#####  Guardamos el dF

In [13]:
filename = 'plebiscito_' + datetime.now().strftime('%m_%d_%Y_%H:%M:%S') + '.xlsx'
# df.to_csv(filename, index = False)

In [14]:
df.to_excel(filename, index = False)

In [26]:
def save_func(df, file_type):
    filename = 'plebiscito_' + datetime.now().strftime('%m_%d_%Y_%H:%M:%S')
    if file_type == 'xlsx':
        df['Time'] = df['Time'].dt.tz_localize(None) # Excel no soporta Datatime con zonas
        filename = filename + '.xlsx'
        df.to_excel(filename, index = False)
    if file_type == 'csv':
        df.replace(',',' ', regex=True, inplace=True)
        df.replace('\n','', regex=True, inplace=True)
        filename = filename + '.csv'
        df.to_csv(filename, index = False)
        

In [27]:
save_func(df, 'csv')